# Feature addition and reduction

As seen in the inferential statistics notebook, there are a lot of multicollinearities between our variables. In this notebook I will try to delete some redundant features and add some features, learning from the EDA, that I think might be helpful for our prediction.

In [152]:
#Load necessary modules
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [153]:
df = pd.read_csv(r'C:\Users\songs\Desktop\Springboard Files\Capstone 2\data\Interim\train1.csv',index_col=0)
df.head()

Store  Dept        Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0     26    92  2011-08-26      87235.57      False        61.10       3.796   
1     34    22  2011-03-25       5945.97      False        53.11       3.480   
2     21    28  2010-12-03       1219.89      False        50.43       2.708   
3      8     9  2010-09-17      11972.71      False        75.32       2.582   
4     19    55  2012-05-18       8271.82      False        58.81       4.029   

   MarkDown1  MarkDown2  MarkDown3  ...  MarkDown5         CPI  Unemployment  \
0       0.00        0.0        0.0  ...       0.00  136.213613         7.767   
1       0.00        0.0        0.0  ...       0.00  128.616064        10.398   
2       0.00        0.0        0.0  ...       0.00  211.265543         8.163   
3       0.00        0.0        0.0  ...       0.00  214.878556         6.315   
4   12613.98        0.0       11.5  ...    3600.79  138.106581         8.150   

     Size  Type_A  Type_B  Type_C  log_revenue  IsMarkDown  Type_barplot  
0  152513       1       0       0    11.431992       False             A  
1  158114       1       0       0     9.299807       False             A  
2  140167       0       1       0     8.733889       False             B  
3  155078       1       0       0     9.738769       False             A  
4  203819       1       0       0     9.492640       False             A  

[5 rows x 21 columns]

In [154]:
df.describe()

Store           Dept   Weekly_Sales    Temperature  \
count  282450.000000  282450.000000  282450.000000  282450.000000   
mean       22.193146      44.286274   15983.503944      60.113598   
std        12.782156      30.503610   22661.098250      18.446505   
min         1.000000       1.000000   -1750.000000      -2.060000   
25%        11.000000      18.000000    2079.337500      46.780000   
50%        22.000000      38.000000    7616.620000      62.150000   
75%        33.000000      74.000000   20245.812500      74.290000   
max        45.000000      99.000000  693099.360000     100.140000   

          Fuel_Price      MarkDown1      MarkDown2      MarkDown3  \
count  282450.000000  282450.000000  282450.000000  282450.000000   
mean        3.360301    2578.777540     872.126294     459.273032   
std         0.458603    6023.733269    5077.429516    5475.263454   
min         2.472000       0.000000    -265.760000     -29.100000   
25%         2.932000       0.000000       0.000000       0.000000   
50%         3.452000       0.000000       0.000000       0.000000   
75%         3.737000    2789.030000       1.910000       4.420000   
max         4.468000   88646.760000  104519.540000  141630.610000   

           MarkDown4     MarkDown5            CPI   Unemployment  \
count  282450.000000  282450.00000  282450.000000  282450.000000   
mean     1077.245617    1659.48530     171.207961       7.968076   
std      3874.121495    4252.38421      39.160786       1.868035   
min         0.000000       0.00000     126.064000       3.879000   
25%         0.000000       0.00000     132.022667       6.891000   
50%         0.000000       0.00000     182.350989       7.866000   
75%       423.220000    2152.40000     212.464799       8.572000   
max     67474.850000  108519.28000     227.232807      14.313000   

                Size         Type_A         Type_B         Type_C  \
count  282450.000000  282450.000000  282450.000000  282450.000000   
mean   136729.826904       0.511800       0.387371       0.100828   
std     61002.286891       0.499862       0.487150       0.301102   
min     34875.000000       0.000000       0.000000       0.000000   
25%     93638.000000       0.000000       0.000000       0.000000   
50%    140167.000000       1.000000       0.000000       0.000000   
75%    202505.000000       1.000000       1.000000       0.000000   
max    219622.000000       1.000000       1.000000       1.000000   

         log_revenue  
count  282450.000000  
mean        9.571147  
std         0.822145  
min         8.083310  
25%         8.863514  
50%         9.441973  
75%        10.136017  
max        13.456102

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282450 entries, 0 to 282450
Data columns (total 21 columns):
Store           282450 non-null int64
Dept            282450 non-null int64
Date            282450 non-null object
Weekly_Sales    282450 non-null float64
IsHoliday       282450 non-null bool
Temperature     282450 non-null float64
Fuel_Price      282450 non-null float64
MarkDown1       282450 non-null float64
MarkDown2       282450 non-null float64
MarkDown3       282450 non-null float64
MarkDown4       282450 non-null float64
MarkDown5       282450 non-null float64
CPI             282450 non-null float64
Unemployment    282450 non-null float64
Size            282450 non-null int64
Type_A          282450 non-null int64
Type_B          282450 non-null int64
Type_C          282450 non-null int64
log_revenue     282450 non-null float64
IsMarkDown      282450 non-null bool
Type_barplot    282450 non-null object
dtypes: bool(2), float64(11), int64(6), object(2)
memory usage: 43.6+

## Combine the Markdown Columns

As seen in the EDA notebook, the markdown columns are highly correlated with each other and a large source of multicollinearity. By combining them into one "Total_Markdown" column, we reduce collinearity. We can also eliminate the IsMarkDown column, since a total markdown of 0 would mean that there is no markdown that week.

In [156]:
df_totalmd = df.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','IsMarkDown'],axis=1)
df_totalmd['Total_MarkDown'] = df[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].sum(axis=1)
df_totalmd.head()

Store  Dept        Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0     26    92  2011-08-26      87235.57      False        61.10       3.796   
1     34    22  2011-03-25       5945.97      False        53.11       3.480   
2     21    28  2010-12-03       1219.89      False        50.43       2.708   
3      8     9  2010-09-17      11972.71      False        75.32       2.582   
4     19    55  2012-05-18       8271.82      False        58.81       4.029   

          CPI  Unemployment    Size  Type_A  Type_B  Type_C  log_revenue  \
0  136.213613         7.767  152513       1       0       0    11.431992   
1  128.616064        10.398  158114       1       0       0     9.299807   
2  211.265543         8.163  140167       0       1       0     8.733889   
3  214.878556         6.315  155078       1       0       0     9.738769   
4  138.106581         8.150  203819       1       0       0     9.492640   

  Type_barplot  Total_MarkDown  
0            A            0.00  
1            A            0.00  
2            B            0.00  
3            A            0.00  
4            A        17931.55

## Delete Type_barplot

I had originally created this column for easier plotting in seaborn, but now that we're not doing plotting, we can delete the column. 

Type of store is a categorical variable, and when doing regression, it's important to have a baseline for categorical variables so that we can interpret the effect of other options in the category. I chose type B as the baseline because it is in the middle in terms of size, and it's be easy to see positive effects (Type A) or negative effects (Type C)

In [157]:
df_nob = df_totalmd.drop(['Type_barplot','Type_B'],axis=1)
df_nob.head()

Store  Dept        Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0     26    92  2011-08-26      87235.57      False        61.10       3.796   
1     34    22  2011-03-25       5945.97      False        53.11       3.480   
2     21    28  2010-12-03       1219.89      False        50.43       2.708   
3      8     9  2010-09-17      11972.71      False        75.32       2.582   
4     19    55  2012-05-18       8271.82      False        58.81       4.029   

          CPI  Unemployment    Size  Type_A  Type_C  log_revenue  \
0  136.213613         7.767  152513       1       0    11.431992   
1  128.616064        10.398  158114       1       0     9.299807   
2  211.265543         8.163  140167       0       0     8.733889   
3  214.878556         6.315  155078       1       0     9.738769   
4  138.106581         8.150  203819       1       0     9.492640   

   Total_MarkDown  
0            0.00  
1            0.00  
2            0.00  
3            0.00  
4        17931.55

## Deciphering Year, Month, Week of Year, and Day

The year, month, week, and day could have an impact on the revenue made. As of now, the date column won't work with a lot of models. Thus, I'm going to parse the date column into year, month week of year, and day columns.

In [158]:
df_parsedate = df_nob
df_parsedate['Date'] = df_parsedate['Date'].astype('datetime64')
df_parsedate['Year'] = df_nob['Date'].dt.year
df_parsedate['Month'] = df_nob['Date'].dt.month
df_parsedate['Week_of_year'] = df_nob['Date'].dt.weekofyear
df_parsedate['Day'] = df_nob['Date'].dt.day
df_parsedate.head()

Store  Dept       Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0     26    92 2011-08-26      87235.57      False        61.10       3.796   
1     34    22 2011-03-25       5945.97      False        53.11       3.480   
2     21    28 2010-12-03       1219.89      False        50.43       2.708   
3      8     9 2010-09-17      11972.71      False        75.32       2.582   
4     19    55 2012-05-18       8271.82      False        58.81       4.029   

          CPI  Unemployment    Size  Type_A  Type_C  log_revenue  \
0  136.213613         7.767  152513       1       0    11.431992   
1  128.616064        10.398  158114       1       0     9.299807   
2  211.265543         8.163  140167       0       0     8.733889   
3  214.878556         6.315  155078       1       0     9.738769   
4  138.106581         8.150  203819       1       0     9.492640   

   Total_MarkDown  Year  Month  Week_of_year  Day  
0            0.00  2011      8            34   26  
1            0.00  2011      3            12   25  
2            0.00  2010     12            48    3  
3            0.00  2010      9            37   17  
4        17931.55  2012      5            20   18

In [159]:
df_parsedate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282450 entries, 0 to 282450
Data columns (total 18 columns):
Store             282450 non-null int64
Dept              282450 non-null int64
Date              282450 non-null datetime64[ns]
Weekly_Sales      282450 non-null float64
IsHoliday         282450 non-null bool
Temperature       282450 non-null float64
Fuel_Price        282450 non-null float64
CPI               282450 non-null float64
Unemployment      282450 non-null float64
Size              282450 non-null int64
Type_A            282450 non-null int64
Type_C            282450 non-null int64
log_revenue       282450 non-null float64
Total_MarkDown    282450 non-null float64
Year              282450 non-null int64
Month             282450 non-null int64
Week_of_year      282450 non-null int64
Day               282450 non-null int64
dtypes: bool(1), datetime64[ns](1), float64(7), int64(9)
memory usage: 39.1 MB


## Adding the Yearly Median value of each Store-Dept combination

This would provide a baseline value for each store-department combination that the models could modify for each week given the other attributes. I'm using the median instead of the mean because the mean can be biased by extreme values, and thus won't give as clear an indicator of each store-department's average performance.

In [181]:
df_groupby = df_parsedate.groupby(['Store','Dept','Year'])['Weekly_Sales'].median()
df_median = df_parsedate.merge(df_groupby, on=['Store','Dept','Year'], how='outer')
df_median.rename(columns={'Weekly_Sales_x':'Weekly_Sales','Weekly_Sales_y':'Median_Sales'}, inplace=True)
df_median.head()

Store  Dept       Date  Weekly_Sales  IsHoliday  Temperature  Fuel_Price  \
0     26    92 2011-08-26      87235.57      False        61.10       3.796   
1     26    92 2011-09-23      80254.53      False        50.72       3.758   
2     26    92 2011-05-27      77715.58      False        56.74       4.034   
3     26    92 2011-07-01      83599.56      False        59.89       3.815   
4     26    92 2011-04-22      77876.10      False        33.81       3.919   

          CPI  Unemployment    Size  Type_A  Type_C  log_revenue  \
0  136.213613         7.767  152513       1       0    11.431992   
1  136.367000         7.767  152513       1       0    11.353279   
2  134.767774         7.818  152513       1       0    11.323042   
3  135.446800         7.767  152513       1       0    11.391769   
4  134.357100         7.818  152513       1       0    11.324981   

   Total_MarkDown  Year  Month  Week_of_year  Day  Median_Sales  
0             0.0  2011      8            34   26     80340.795  
1             0.0  2011      9            38   23     80340.795  
2             0.0  2011      5            21   27     80340.795  
3             0.0  2011      7            26    1     80340.795  
4             0.0  2011      4            16   22     80340.795

In [182]:
df_median.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282450 entries, 0 to 282449
Data columns (total 19 columns):
Store             282450 non-null int64
Dept              282450 non-null int64
Date              282450 non-null datetime64[ns]
Weekly_Sales      282450 non-null float64
IsHoliday         282450 non-null bool
Temperature       282450 non-null float64
Fuel_Price        282450 non-null float64
CPI               282450 non-null float64
Unemployment      282450 non-null float64
Size              282450 non-null int64
Type_A            282450 non-null int64
Type_C            282450 non-null int64
log_revenue       282450 non-null float64
Total_MarkDown    282450 non-null float64
Year              282450 non-null int64
Month             282450 non-null int64
Week_of_year      282450 non-null int64
Day               282450 non-null int64
Median_Sales      282450 non-null float64
dtypes: bool(1), datetime64[ns](1), float64(8), int64(9)
memory usage: 41.2 MB


## Adding some Date variables

Some models don't work well with the dates, so I'm going to see if any dates have a larger impact than others.

From from the >$300,000 table in the EDA notebook, it appears that a signficant portion of them are made in the thanksgiving week, so we'll make a new feature called "IsThanksgiving" to distinguish those values made during Thanksgiving weeks. Looking at the average sales per week, it appears that sales are higher near Christmas, so I'll also create a new feature called "IsChristmas". After this, we could delete the Date Column.

In [183]:
df_bigholidays = df_median
df_bigholidays['IsThanksgiving'] = (df_bigholidays['Date'] == '2011-11-25') | (df_bigholidays['Date'] == '2010-11-26')
df_bigholidays['IsChristmas'] = (df_bigholidays['Date'] == '2010-12-31') | (df_bigholidays['Date'] == '2011-12-30')
df_bigholidays = df_bigholidays.drop('Date',axis=1)
df_bigholidays.head()

Store  Dept  Weekly_Sales  IsHoliday  Temperature  Fuel_Price         CPI  \
0     26    92      87235.57      False        61.10       3.796  136.213613   
1     26    92      80254.53      False        50.72       3.758  136.367000   
2     26    92      77715.58      False        56.74       4.034  134.767774   
3     26    92      83599.56      False        59.89       3.815  135.446800   
4     26    92      77876.10      False        33.81       3.919  134.357100   

   Unemployment    Size  Type_A  Type_C  log_revenue  Total_MarkDown  Year  \
0         7.767  152513       1       0    11.431992             0.0  2011   
1         7.767  152513       1       0    11.353279             0.0  2011   
2         7.818  152513       1       0    11.323042             0.0  2011   
3         7.767  152513       1       0    11.391769             0.0  2011   
4         7.818  152513       1       0    11.324981             0.0  2011   

   Month  Week_of_year  Day  Median_Sales  IsThanksgiving  IsChristmas  
0      8            34   26     80340.795           False        False  
1      9            38   23     80340.795           False        False  
2      5            21   27     80340.795           False        False  
3      7            26    1     80340.795           False        False  
4      4            16   22     80340.795           False        False

In [184]:
df_bigholidays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282450 entries, 0 to 282449
Data columns (total 20 columns):
Store             282450 non-null int64
Dept              282450 non-null int64
Weekly_Sales      282450 non-null float64
IsHoliday         282450 non-null bool
Temperature       282450 non-null float64
Fuel_Price        282450 non-null float64
CPI               282450 non-null float64
Unemployment      282450 non-null float64
Size              282450 non-null int64
Type_A            282450 non-null int64
Type_C            282450 non-null int64
log_revenue       282450 non-null float64
Total_MarkDown    282450 non-null float64
Year              282450 non-null int64
Month             282450 non-null int64
Week_of_year      282450 non-null int64
Day               282450 non-null int64
Median_Sales      282450 non-null float64
IsThanksgiving    282450 non-null bool
IsChristmas       282450 non-null bool
dtypes: bool(3), float64(8), int64(9)
memory usage: 39.6 MB


## Adding department 72

Looking at the EDA, it appears that department 72 has sales higher than others. I'll create an "IsDept72" Column, then.

In [185]:
df_dept72 = df_bigholidays
df_dept72['IsDept72'] = (df_bigholidays['Dept'] == 72)
df_dept72.head()

Store  Dept  Weekly_Sales  IsHoliday  Temperature  Fuel_Price         CPI  \
0     26    92      87235.57      False        61.10       3.796  136.213613   
1     26    92      80254.53      False        50.72       3.758  136.367000   
2     26    92      77715.58      False        56.74       4.034  134.767774   
3     26    92      83599.56      False        59.89       3.815  135.446800   
4     26    92      77876.10      False        33.81       3.919  134.357100   

   Unemployment    Size  Type_A  ...  log_revenue  Total_MarkDown  Year  \
0         7.767  152513       1  ...    11.431992             0.0  2011   
1         7.767  152513       1  ...    11.353279             0.0  2011   
2         7.818  152513       1  ...    11.323042             0.0  2011   
3         7.767  152513       1  ...    11.391769             0.0  2011   
4         7.818  152513       1  ...    11.324981             0.0  2011   

   Month  Week_of_year  Day  Median_Sales  IsThanksgiving  IsChristmas  \
0      8            34   26     80340.795           False        False   
1      9            38   23     80340.795           False        False   
2      5            21   27     80340.795           False        False   
3      7            26    1     80340.795           False        False   
4      4            16   22     80340.795           False        False   

   IsDept72  
0     False  
1     False  
2     False  
3     False  
4     False  

[5 rows x 21 columns]

In [186]:
def fit_model(df):
    #Creating Features and Target
    X = df.drop(['Weekly_Sales','log_revenue'], axis=1).values
    y = df['Weekly_Sales'].values

    #Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    #Create the Regression model:
    linear = LinearRegression()
    linear.fit(X_train, y_train)

    #Making predictions
    y_pred = linear.predict(X_test)
    
    y_true = y_test
    
    print(mean_absolute_error(y_true, y_pred))
    
fit_model(df_dept72)

2591.2742359765675


In [190]:
df_dept72.to_csv(r'C:\Users\songs\Desktop\Springboard Files\Capstone 2\data\Interim\train_all_features.csv')